In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv")
test=pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv")

# EDA

In [ ]:
pip install dataprep

In [ ]:
import pandas as pd
from dataprep.eda import create_report

# df = pd.read_csv("train.csv")
create_report(train)

# Understanding Data

In [ ]:
train.info()

In [ ]:
features=train.columns[:-1]

In [ ]:
col_f=[]
col_o=[]

for i in train[features].columns:
    if train[i].dtype=='float64':
        col_f.append(i)
    else:
        col_o.append(i)
        
print("integer column:\n ",col_f,'\n')
print("string column: \n",col_o)

In [ ]:
x_train=train[features]
y_train=train["Made_Purchase"]

In [ ]:
y_train.value_counts()

# Preprocessing

In [ ]:
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import normalize

encode= OrdinalEncoder()
Kimputer=KNNImputer(n_neighbors=7, weights="uniform")

col_n=["LandingPage_Duration",'HomePage_Duration','ProductDescriptionPage_Duration','GoogleMetric:Bounce Rates','GoogleMetric:Exit Rates','GoogleMetric:Page Values']

def process(dt):
    dt[col_o]=pd.DataFrame(encode.fit_transform(dt[col_o]))
    dt[dt.columns]=Kimputer.fit_transform(dt)
    
    dt[col_n]=normalize(dt[col_n], norm='l2', axis=1, copy=True, return_norm=False)

In [ ]:
process(x_train)
x_train

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
target_label = label_encoder.fit_transform(y_train)

In [ ]:
target_label[:5]

In [ ]:
x_test=test[features]
process(x_test)
x_test

# Removing Outliers

In [ ]:

# # calculate upper and lower bounds for outlier detection
# Q1 = x_train.quantile(0.25)
# Q3 = x_train.quantile(0.75)
# IQR = Q3 - Q1
# lower_bound = Q1 - 1.5 * IQR
# upper_bound = Q3 + 1.5 * IQR

# # identify and mark the outliers as NaN
# x_train[(x_train < lower_bound) | (x_train > upper_bound)] = np.nan
# process(x_train)

In [ ]:

# # calculate upper and lower bounds for outlier detection
# Q1 = x_test.quantile(0.25)
# Q3 = x_test.quantile(0.75)
# IQR = Q3 - Q1
# lower_bound = Q1 - 1.5 * IQR
# upper_bound = Q3 + 1.5 * IQR

# # identify and mark the outliers as NaN
# x_test[(x_test < lower_bound) | (x_test > upper_bound)] = np.nan
# process(x_test)

# Feature Selection

In [ ]:
from sklearn.feature_selection import SelectKBest,chi2

#apply SelectKBest class to extract top 15 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(x_train,y_train)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(x_train.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
featureScores

print(featureScores.nlargest(15,'Score'))  #print 10 best features
new_x_train=bestfeatures.transform(x_train)

# Splitting and Validation Set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(new_x_train,target_label, test_size=0.2,random_state=42, stratify=y_train)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
from sklearn.metrics import roc_auc_score, ConfusionMatrixDisplay, confusion_matrix

def get_metrics(classifier, Xvalid, yvalid):
    print("Validation Score : ", classifier.score(Xvalid, yvalid))
    
    y_train_pred = classifier.predict_proba(X_train)[:,1]
    y_valid_pred = classifier.predict_proba(Xvalid)[:,1]
    
    print("AUC Train: {:.4f}\nAUC Valid: {:.4f}".format(roc_auc_score(y_train, y_train_pred),roc_auc_score(yvalid, y_valid_pred)))
    
    y_valid_predict = classifier.predict(Xvalid)
    
    #Confusion Matrix
    cm = confusion_matrix(yvalid,y_valid_predict)
    dist = ConfusionMatrixDisplay(cm)
    dist.plot()
    plt.show()

# Dummy

In [ ]:
# from sklearn.dummy import DummyClassifier
# clf = DummyClassifier(strategy="most_frequent")
# clf.fit(x_train, y_train)
# y_pred = clf.predict(x_train)
# from sklearn.metrics import f1_score
# f1_score(y_train,y_pred)

# Logistic Regression
### highest score 0.67 without removing outliers

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.linear_model import LogisticRegression
# clf = LogisticRegression(random_state=64, solver='newton-cg')
# clf.fit(X_train, y_train)
# print("Training Score : ", clf.score(X_train,y_train))

# # get matrix
# get_metrics(clf, X_valid,y_valid)

# Random Forest

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# model=RandomForestClassifier()
# model.fit(X_train,y_train)
# print("Training Score : ", model.score(X_train,y_train))

# # get matrix
# get_metrics(model, X_valid,y_valid)

# Gradient Boosting
### score 0.58 with removing outliers

In [ ]:
# from sklearn.ensemble import GradientBoostingClassifier
# clf = GradientBoostingClassifier(n_estimators=100)

# # train the model on the training data
# clf.fit(X_train, y_train)
# print("Training Score : ", model.score(X_train,y_train))

# # get matrix
# get_metrics(model, X_valid,y_valid)

# Disicion Tree

In [ ]:
# from sklearn.tree import DecisionTreeClassifier
# x_test=test
# clf = DecisionTreeClassifier()

# # train the model on the training data
# clf.fit(X_train, y_train)

# # make predictions on the test data
# print("Training Score : ", clf.score(X_train,y_train))

# # get matrix
# get_metrics(clf, X_valid,y_valid)

# SVM
### Score 0.59266

In [ ]:
# from sklearn.svm import SVC


In [ ]:
# x_test=test
# clf = SVC(kernel='rbf')



In [ ]:
# train the model on the training data
# clf.fit(X_train, y_train)



In [ ]:
# # make predictions on the test data
# print("Training Score : ", clf.score(X_train,y_train))

# # get matrix
# get_metrics(clf, X_valid,y_valid)

# KNN

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# clf = KNeighborsClassifier(n_neighbors=5)

# # train the model on the training data
# clf.fit(X_train, y_train)

# # make predictions on the test data
# print("Training Score : ", clf.score(X_train,y_train))

# # get matrix
# get_metrics(clf, X_valid,y_valid)

# XGBClassifier

In [ ]:
# from xgboost import XGBClassifier
# xgb = XGBClassifier()
# xgb.fit(X_train, y_train)

# print("Training Score : ", xgb.score(X_train,y_train))

# # get matrix
# get_metrics(xgb, X_valid,y_valid)

# Hypertunning

In [ ]:
# params = {
#     'n_estimators': [100, 200, 300, 400, 500],
#     'max_samples': [0.5, 0.7, 1.0],
#     'max_features': [0.5, 0.7, 1.0],
#     'bootstrap': [True, False]
# }
params ={
    'max_depth': [3, 4, 5, 6, 8, 10, 12, 15],
    'learning_rate': [0.05,0.10,0.15,0.20,0.25,0.30],
    'n_estimators': [100, 200,300,400,500,600,900,1000,2000,4000,5000],
    'colsample_bytree': [0.3,0.4,0.5,0.7],
    'gamma':[0.0,0.1,0.2,0.3,0.4],
    'min_child_weight': [1,3,5,7,8],
    'subsample':[0.5,0.6,0.7,0.8,0.9,1],
    'reg_alpha':[0, 0.001, 0.01, 0.03, 0.08, 0.3, 0.5],
    'reg_lambda':[0, 0.001, 0.01, 0.03, 0.08, 0.3, 0.5],
    'scale_pos_weight':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],
    'max_delta_step':[0,1,2,3,4,5,6,7,8,9,10],
    'base_score': [0.5,0.7,0.9]
}

## XGB Classifier

In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
# from sklearn.model_selection import GridSearchCV
xgb_tunned =XGBClassifier()
# bagging = BaggingClassifier(xgb_tunned)


random_search = RandomizedSearchCV(xgb_tunned, params, n_iter=5, scoring='roc_auc', n_jobs=-1, cv=5, verbose = 3)

random_search.fit(X_train, y_train)
print("Training Score: ", random_search.score(X_train, y_train))
get_metrics(random_search, X_train, y_train)

print(random_search.best_params_)

In [ ]:
# param_grid = {
#     'max_depth': [8,4],
#     'min_child_weight': [1,3,5,7,8],
#     'learning_rate': [0.05,0.10,0.15,0.20,0.25,0.30],
#     'colsample_bytree': [0.3,0.4,0.5,0.7],
# }

# # create an instance of GridSearchCV
# grid_search = GridSearchCV(xgb_tunned, param_grid, scoring = 'roc_auc',n_jobs=-1,cv=5,verbose=3)
# grid_search.fit(X_train,y_train)

In [ ]:
model_xgboost = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6,
              enable_categorical=False, gamma=0.4, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.3, max_delta_step=10, max_depth=3,
              min_child_weight=47, monotone_constraints='()',
              n_estimators=1000, n_jobs=12, num_parallel_tree=1,
              predictor='auto', random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1.2, subsample=0.7, tree_method='exact',
              validate_parameters=1, verbosity=None)

eval_set = [(X_valid, y_valid)]

model_xgboost.fit(X_train,y_train,early_stopping_rounds=10,eval_set=eval_set,verbose=True)

print("Training Score: ", model_xgboost.score(X_valid,y_valid))
get_metrics(model_xgboost, X_valid, y_valid)

In [ ]:
# from sklearn.model_selection import cross_val_score
# score=cross_val_score(model_xgboost,x_train,target_label,cv=10)
# print(score)

# score.mean()

# Predicting for x_test

In [ ]:
final_test=bestfeatures.transform(x_test)
prediction = model_xgboost.predict(final_test)


In [ ]:
predict=[]
for i in prediction:
    if i==1:
        predict.append(True)
    else:
        predict.append(False)

In [ ]:
sample=pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/sample.csv")

In [ ]:
submission=pd.DataFrame(columns=['id', 'Made_Purchase'])
submission['id']=[i for i in range(len(predict))]
submission["Made_Purchase"]=predict
submission.to_csv('submission.csv',index=False)